# ECON 5140: Applied Econometrics
## Homework 2: Regression/ETS Models & ARIMA Models

**Covers:** Lesson 3 (Regression and ETS) & Lesson 4 (ARIMA Models)

---

# PART 1: REGRESSION AND ETS MODELS

## Section A: Analytical Problems (Regression & ETS)

### Problem 1: Simple Exponential Smoothing (SES)

A coffee shop tracks daily sales. The current level estimate is ℓ₉ = 250 (cups sold). 
On day 10, actual sales are y₁₀ = 280 cups.

The shop uses Simple Exponential Smoothing with smoothing parameter α = 0.3.

**Questions:**

a) Calculate the updated level estimate ℓ₁₀ using: ℓₜ = αyₜ + (1-α)ℓₜ₋₁

b) What was the one-step-ahead forecast error on day 10?

c) Calculate the forecast for day 11: ŷ₁₁|₁₀

d) Rewrite the SES update as an error correction model and verify your answer to part (a).

e) If the smoothing parameter were increased to α = 0.7, would the new level estimate ℓ₁₀ be higher or lower? Why?


### Problem 2: Holt's Linear Trend Method

A streaming service tracks monthly subscriber growth. At the end of month 5:
- Level estimate: ℓ₅ = 10,000 subscribers
- Trend estimate: b₅ = 500 subscribers/month

In month 6, actual subscribers: y₆ = 11,200

The model uses: α = 0.4 (level smoothing), β = 0.2 (trend smoothing)

**Holt's equations:**
- ℓₜ = αyₜ + (1-α)(ℓₜ₋₁ + bₜ₋₁)
- bₜ = β(ℓₜ - ℓₜ₋₁) + (1-β)bₜ₋₁

**Questions:**

a) Calculate the updated level ℓ₆

b) Calculate the updated trend b₆

c) Generate forecasts for months 7 and 8:
   - ŷ₇|₆ = ℓ₆ + b₆
   - ŷ₈|₆ = ℓ₆ + 2b₆

### Problem 3: Forecast Evaluation Metrics

A forecaster generates 5 one-step-ahead forecasts:

| Period | Actual (yₜ) | Forecast (ŷₜ) | Error (eₜ) |
|--------|-------------|---------------|------------|
| 1      | 100         | 95            | 5          |
| 2      | 110         | 108           | 2          |
| 3      | 105         | 112           | -7         |
| 4      | 115         | 110           | 5          |
| 5      | 120         | 125           | -5         |

**Questions:**

a) Calculate the Mean Absolute Error (MAE): MAE = (1/n)Σ|eₜ|

b) Calculate the Mean Squared Error (MSE): MSE = (1/n)Σeₜ²

c) Calculate the Root Mean Squared Error (RMSE): RMSE = √MSE

d) Calculate the Mean Absolute Percentage Error (MAPE): MAPE = (1/n)Σ|eₜ/yₜ|×100%

e) Based on RMSE, which forecast error was most problematic? Why does RMSE penalize it more than MAE?

### Problem 4: MA(1) Model

Consider an MA(1) model: yₜ = εₜ + θεₜ₋₁

where εₜ ~ N(0, σ²) and θ = 0.6

At time t = 10:
- ε₁₀ = 2
- ε₉ = -1

**Questions:**

a) Calculate y₁₀

b) Calculate the one-step-ahead forecast ŷ₁₁|₁₀
   - Hint: E[ε₁₁] = 0

c) Calculate the two-step-ahead forecast ŷ₁₂|₁₀

d) Why does the MA(1) forecast return to zero after one step?

e) Calculate the lag-1 autocorrelation: ρ(1) = θ/(1 + θ²)

# PART 3: CODING PROBLEMS

## Problem 1: Comprehensive Time Series Forecasting
You will work with two datasets: retail sales (ETS/Regression) and stock returns (ARIMA).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)


# ====================================================================
# DATASET 1: RETAIL SALES (for Regression & ETS)
# ====================================================================
print("\n" + "=" * 70)
print("DATASET 1: Monthly Retail Sales")
print("=" * 70)

# Create 4 years of monthly data
dates = pd.date_range('2021-01-01', '2024-12-31', freq='MS')
n_months = len(dates)
t = np.arange(n_months)

# Components
trend = 1000 + 15*t  # Growing business
yearly_seasonal = 300 * np.sin(2*np.pi*t/12) + 200 * np.cos(2*np.pi*t/12)

# Holiday effects (November-December spike)
holiday_effect = np.zeros(n_months)
for year in range(4):
    nov_idx = year*12 + 10  # November
    dec_idx = year*12 + 11  # December
    if nov_idx < n_months:
        holiday_effect[nov_idx] = 400
    if dec_idx < n_months:
        holiday_effect[dec_idx] = 600

# Random noise
noise = np.random.normal(0, 80, n_months)

# Combine
sales = trend + yearly_seasonal + holiday_effect + noise
sales = np.maximum(sales, 0)

# Create DataFrame
df_sales = pd.DataFrame({
    'Date': dates,
    'Sales': sales,
    'Month': dates.month,
    'Year': dates.year,
    'Time': t
})
df_sales.set_index('Date', inplace=True)

print(f"Date range: {df_sales.index[0].date()} to {df_sales.index[-1].date()}")
print(f"Number of months: {len(df_sales)}")
print(f"\nSales Statistics:")
print(df_sales['Sales'].describe())

# ====================================================================
# PART A: REGRESSION AND ETS MODELS
# ====================================================================
print("\n" + "=" * 70)
print("PART A: REGRESSION AND ETS MODELS")
print("=" * 70)

# --------------------------------------------------------------------
# A1: Time Series Visualization
# --------------------------------------------------------------------
print("\n" + "-" * 70)
print("A1: Exploratory Visualization")
print("-" * 70)

# YOUR CODE:
# 1. Create time series plot of monthly sales
#    - Full 4-year series
#    - Add title and labels
#
# 2. Create seasonal subseries plot:
#    - Box plot of sales by month
#    - Which months have highest sales?
#
# 3. Create a decomposition plot (conceptual):
#    - Plot with 3 subplots:
#      * Original series
#      * 12-month moving average (trend)
#      * Detrended series (original - trend)

# --------------------------------------------------------------------
# A2: Linear Regression with Trend and Seasonality
# --------------------------------------------------------------------
print("\n" + "-" * 70)
print("A2: Regression Forecasting")
print("-" * 70)

# YOUR CODE:
# 1. Create seasonal dummy variables:
#    - 11 dummies for months (leave January as baseline)
#    - Use pd.get_dummies(df_sales['Month'], drop_first=True)
#
# 2. Fit regression model:
#    Sales = β₀ + β₁(Time) + Σγₘ(Month_m) + ε
#
# 3. Print regression summary
#
# 4. Interpret:
#    - What is the monthly trend (β₁)?
#    - Which month has the largest seasonal effect?
#    - Are the coefficients significant?
#
# 5. Generate fitted values and calculate:
#    - R²
#    - RMSE
#    - MAE

# --------------------------------------------------------------------
# A3: Fourier Terms for Seasonality
# --------------------------------------------------------------------
print("\n" + "-" * 70)
print("A3: Fourier Seasonality")
print("-" * 70)

# YOUR CODE:
# 1. Create Fourier terms for yearly seasonality (m=12):
#    For k=1,2:
#    - sin(2πkt/12)
#    - cos(2πkt/12)
#
# 2. Fit regression:
#    Sales = β₀ + β₁(Time) + Σ[αₖsin(2πkt/12) + βₖcos(2πkt/12)] + ε
#
# 3. Compare with dummy variable model:
#    - R²
#    - AIC
#    - Number of parameters
#
# 4. Plot fitted values from both models on same graph
#
# 5. Which approach do you prefer? Why?
# --------------------------------------------------------------------
# A4: Simple Exponential Smoothing (SES)
# --------------------------------------------------------------------
print("\n" + "-" * 70)
print("A4: Simple Exponential Smoothing")
print("-" * 70)

# Deseasonalize the data first for SES
# Use only first 36 months for training

# YOUR CODE:
# 1. Deseasonalize sales using seasonal averages:
#    - Calculate average sales for each month
#    - Subtract seasonal component
#
# 2. Fit SES on deseasonalized data:
#    - Use SimpleExpSmoothing from statsmodels
#    - Optimize smoothing parameter α
#
# 3. Print:
#    - Optimized α value
#    - What does this value tell you about the series?
#
# 4. Generate forecasts for next 12 months
#
# 5. Add seasonality back to forecasts
#
# 6. Calculate forecast accuracy on holdout period

# --------------------------------------------------------------------
# A5: Holt's Linear Trend Method
# --------------------------------------------------------------------
print("\n" + "-" * 70)
print("A5: Holt's Method")
print("-" * 70)

# YOUR CODE:
# 1. Fit Holt's linear trend method:
#    - Use ExponentialSmoothing with trend='add', seasonal=None
#    - On deseasonalized data
#
# 2. Print optimized parameters:
#    - α (level smoothing)
#    - β (trend smoothing)
#
# 3. Extract final state:
#    - Level (ℓₜ)
#    - Trend (bₜ)
#
# 4. Generate 12-month forecasts
#
# 5. Compare with SES:
#    - Plot both forecast paths
#    - Which captures the trend better?

# --------------------------------------------------------------------
# A6: Holt-Winters Seasonal Method
# --------------------------------------------------------------------
print("\n" + "-" * 70)
print("A6: Holt-Winters Method")
print("-" * 70)

# YOUR CODE:
# 1. Fit Holt-Winters with additive seasonality:
#    - Use ExponentialSmoothing
#    - trend='add', seasonal='add', seasonal_periods=12
#    - Use first 36 months for training
#
# 2. Print optimized parameters:
#    - α, β, γ (seasonal smoothing)
#
# 3. Extract components:
#    - Level
#    - Trend  
#    - Seasonal indices
#
# 4. Plot decomposition from Holt-Winters
#
# 5. Generate 12-month forecasts
#
# 6. Compare forecast accuracy with:
#    - Regression model
#    - SES
#    - Holt's method


DATASET 1: Monthly Retail Sales
Date range: 2021-01-01 to 2024-12-01
Number of months: 48

Sales Statistics:
count      48.000000
mean     1419.409544
std       352.878551
min       722.634428
25%      1196.284168
50%      1393.908200
75%      1710.667034
max      2412.774859
Name: Sales, dtype: float64

PART A: REGRESSION AND ETS MODELS

----------------------------------------------------------------------
A1: Exploratory Visualization
----------------------------------------------------------------------

----------------------------------------------------------------------
A2: Regression Forecasting
----------------------------------------------------------------------

----------------------------------------------------------------------
A3: Fourier Seasonality
----------------------------------------------------------------------

----------------------------------------------------------------------
A5: Simple Exponential Smoothing
--------------------------------------------

In [3]:
# ====================================================================
# PART B: ARIMA MODELS
# ====================================================================
print("\n" + "=" * 70)
print("PART B: ARIMA MODELS")
print("=" * 70)

# ====================================================================
# DATASET 2: DAILY STOCK RETURNS (for ARIMA)
# ====================================================================
print("\n" + "=" * 70)
print("DATASET 2: Daily Stock Returns")
print("=" * 70)

# Create 500 days of stock price data
n_days = 500
dates_stock = pd.date_range('2023-01-01', periods=n_days, freq='D')

# Generate AR(1) returns with some volatility clustering
returns = np.zeros(n_days)
returns[0] = np.random.normal(0, 0.01)

phi = 0.05  # Small autocorrelation in returns
for i in range(1, n_days):
    returns[i] = phi * returns[i-1] + np.random.normal(0, 0.015)

# Calculate price from returns (starting at 100)
price = 100 * np.exp(np.cumsum(returns))

df_stock = pd.DataFrame({
    'Date': dates_stock,
    'Price': price,
    'Returns': returns * 100  # Convert to percentage
})
df_stock.set_index('Date', inplace=True)

print(f"Date range: {df_stock.index[0].date()} to {df_stock.index[-1].date()}")
print(f"Number of days: {len(df_stock)}")
print(f"\nPrice Statistics:")
print(df_stock['Price'].describe())

# --------------------------------------------------------------------
# B1: Stationarity Testing
# --------------------------------------------------------------------
print("\n" + "-" * 70)
print("B1: Stationarity Tests")
print("-" * 70)

# YOUR CODE:
# 1. Visual stationarity check for stock prices:
#    - Plot price series
#    - Plot with rolling mean (30-day window)
#    - Plot with rolling std (30-day window)
#    - Does it look stationary?
#
# 2. Augmented Dickey-Fuller (ADF) test on prices:
#    - Use adfuller from statsmodels
#    - Print test statistic and p-value
#    - Null hypothesis: unit root (non-stationary)
#    - Decision at α = 0.05?
#
# 3. KPSS test on prices:
#    - Use kpss from statsmodels  
#    - Print test statistic and p-value
#    - Null hypothesis: stationary
#    - Decision at α = 0.05?
#
# 4. Interpret both tests together:
#    - Do they agree?
#    - Is the price series stationary?

# --------------------------------------------------------------------
# B2: First Differencing
# --------------------------------------------------------------------
print("\n" + "-" * 70)
print("B2: Differencing for Stationarity")
print("-" * 70)

# YOUR CODE:
# 1. Calculate first differences of prices:
#    diff_price = df_stock['Price'].diff()
#
# 2. Plot the differenced series
#
# 3. Run ADF test on differenced series:
#    - Report results
#    - Is it now stationary?
#
# 4. Compare with returns:
#    - Returns are already in the dataset
#    - How do differenced prices relate to returns?
#    - Calculate correlation
#
# 5. Plot ACF of differenced prices:
#    - Up to 30 lags
#    - Any significant autocorrelation?

# --------------------------------------------------------------------
# B3: ACF and PACF Analysis
# --------------------------------------------------------------------
print("\n" + "-" * 70)
print("B3: ACF and PACF")
print("-" * 70)

# YOUR CODE:
# Work with returns (already stationary):
#
# 1. Create 2x1 subplot:
#    - ACF of returns (40 lags)
#    - PACF of returns (40 lags)
#
# 2. Interpret the plots:
#    - Are there significant spikes?
#    - At which lags?
#    - Does ACF decay gradually or cut off?
#    - Does PACF decay gradually or cut off?
#
# 3. Based on patterns, suggest:
#    - AR order (p)?
#    - MA order (q)?
#
# 4. Calculate specific autocorrelations manually:
#    - ρ(1) = Corr(returns_t, returns_{t-1})
#    - ρ(5) = Corr(returns_t, returns_{t-5})

# --------------------------------------------------------------------
# B4: AR Model Estimation
# --------------------------------------------------------------------
print("\n" + "-" * 70)
print("B4: AR(p) Model")
print("-" * 70)

# YOUR CODE:
# 1. Fit AR(1) model on returns:
#    - Use ARIMA(1,0,0)
#    - Print summary
#
# 2. Extract and interpret:
#    - ϕ₁ coefficient
#    - Is it statistically significant?
#    - What does the sign tell you?
#
# 3. Check stationarity condition:
#    - Is |ϕ₁| < 1?
#
# 4. Fit AR(2) and AR(3):
#    - Compare AIC values
#    - Which model is preferred?
#
# 5. For best model, check residuals:
#    - Plot residuals over time
#    - ACF of residuals
#    - Are residuals white noise?

# --------------------------------------------------------------------
# B5: MA Model Estimation
# --------------------------------------------------------------------
print("\n" + "-" * 70)
print("B5: MA(q) Model")
print("-" * 70)

# YOUR CODE:
# 1. Fit MA(1) model on returns:
#    - Use ARIMA(0,0,1)
#
# 2. Print summary and interpret:
#    - θ₁ coefficient
#    - Significance
#
# 3. Check invertibility:
#    - Is |θ₁| < 1?
#
# 4. Compare MA(1) vs AR(1):
#    - AIC
#    - BIC
#    - Log-likelihood
#    - Which fits better?
#
# 5. Residual diagnostics for MA(1)

# --------------------------------------------------------------------
# B6: ARMA Model
# --------------------------------------------------------------------
print("\n" + "-" * 70)
print("B6: ARMA(p,q) Model")
print("-" * 70)

# YOUR CODE:
# 1. Fit ARMA(1,1) on returns:
#    - Use ARIMA(1,0,1)
#
# 2. Compare with AR(1) and MA(1):
#    - Does ARMA(1,1) improve fit?
#    - Check AIC
#
# 3. Test for overparameterization:
#    - Are both ϕ₁ and θ₁ significant?
#    - If not, which simpler model is better?
#
# 4. Create comparison table:
#    | Model    | AIC | BIC | Log-Lik | Parameters |

# --------------------------------------------------------------------
# B7: Automatic ARIMA Selection
# --------------------------------------------------------------------
print("\n" + "-" * 70)
print("B7: Automatic ARIMA")
print("-" * 70)

# YOUR CODE:
# 1. Use auto_arima or stepwise selection:
#    - Search over p ∈ [0,5], q ∈ [0,5]
#    - Use AIC for selection
#    - Note: You may need to implement simple grid search
#    
#    Example structure:
#    best_aic = np.inf
#    best_model = None
#    for p in range(6):
#        for q in range(6):
#            try:
#                model = ARIMA(returns, order=(p,0,q))
#                fitted = model.fit()
#                if fitted.aic < best_aic:
#                    best_aic = fitted.aic
#                    best_model = (p,0,q)
#            except:
#                continue
#
# 2. Report best model specification
#
# 3. Fit and summarize best model
#
# 4. Does automatic selection match your manual identification?

# --------------------------------------------------------------------
# B8: Forecasting with ARIMA
# --------------------------------------------------------------------
print("\n" + "-" * 70)
print("B8: ARIMA Forecasting")
print("-" * 70)

# YOUR CODE:
# Using your best ARIMA model from B7:
#
# 1. Generate forecasts:
#    - 20-day ahead forecasts for returns
#    - Include prediction intervals
#
# 2. Plot forecast:
#    - Last 100 days of actual returns
#    - 20-day forecast
#    - 95% prediction interval
#
# 3. Convert return forecasts to price forecasts:
#    - Starting from last observed price
#    - price_{t+h} = price_t × exp(cumsum(return forecasts))
#
# 4. Observe forecast behavior:
#    - Do return forecasts converge to zero?
#    - How does forecast uncertainty change with horizon?
#    - Why do prediction intervals widen?



PART B: ARIMA MODELS

DATASET 2: Daily Stock Returns
Date range: 2023-01-01 to 2024-05-14
Number of days: 500

Price Statistics:
count    500.000000
mean     115.002874
std       12.793924
min       94.210519
25%      102.898666
50%      115.340500
75%      125.312316
max      149.153080
Name: Price, dtype: float64

----------------------------------------------------------------------
B1: Stationarity Tests
----------------------------------------------------------------------

----------------------------------------------------------------------
B2: Differencing for Stationarity
----------------------------------------------------------------------

----------------------------------------------------------------------
B3: ACF and PACF
----------------------------------------------------------------------

----------------------------------------------------------------------
B4: AR(p) Model
----------------------------------------------------------------------

-------------------